# Exploratory Data Analysis

The first of this notebook will have pre-join exploratory data analysis of the given data.

#Loading all packages and datasets

In [0]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, isnull, percent_rank, avg
from pyspark.sql.functions import min
from pyspark.sql.functions import col, max
from pyspark.sql.functions import format_string
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import concat
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp
from pyspark.sql.functions import expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import instr
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors
from pyspark.mllib.evaluation import MulticlassMetrics
import random as rnd
from pyspark.sql.functions import mean as _mean

from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 1- Prejoin EDA

In [0]:
data_BASE_DIR = "dbfs:/mnt/mids-w261/"
# display(dbutils.fs.ls(f"{data_BASE_DIR}"))

# Inspect the Mount's Final Project folder 
data_BASE_DIR = "dbfs:/mnt/mids-w261/datasets_final_project_2022/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/,parquet_airlines_data/,0,1656618287000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/,parquet_airlines_data_1y/,0,1656630272000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0,1656630114000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0,1656630205000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/,parquet_weather_data/,0,1656622074000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_1y/,parquet_weather_data_1y/,0,1656631614000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m/,parquet_weather_data_3m/,0,1656630651000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_6m/,parquet_weather_data_6m/,0,1656631047000
dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/,stations_data/,0,1656713663000


### Init Script and SAS Token

In [0]:
# ignoring this for now. 
#Initializes blob storage credentials/location
blob_container = "w261-sec4-group2" # The name of your container created in https://portal.azure.com
storage_account = "kdevery" # The name of your Storage account created in https://portal.azure.com
secret_scope = "sec4-group2" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

#Points to SAS token
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
# displaying dataset in our storage blob
display(dbutils.fs.ls(f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"))

path,name,size,modificationTime
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data/,feature_engineered_data/,0,1668924639000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data_test/,feature_engineered_data_test/,0,1668924670000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_train_data/,feature_engineered_train_data/,0,1668559613000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data/,merged_cleaned_data/,0,1669494945000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_test/,merged_cleaned_data_test/,0,1669495012000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_train/,merged_cleaned_data_train/,0,1669495000000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_data/,merged_data/,0,1669494746000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/number_flights_and_delay_rate/,number_flights_and_delay_rate/,0,1669798919000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/pagerank_scores/,pagerank_scores/,0,1669854087000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/premerge_airline_data/,premerge_airline_data/,0,1669567605000


### Loading datasets

In [0]:
df_stations = spark.read.parquet(f"{data_BASE_DIR}stations_data/*")
# display(df_stations)

airline_final_df = spark.read.parquet(f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net/premerge_airline_data")

weather_final_df = spark.read.parquet(f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net/premerge_weather_station_data")
#display(df_weather)

#Stations Dataset - EDA

In [0]:
# we have 2229 unique airport codes in the df_stations dataset. 
print(f" Total number of aiport codes :{df_stations.select('neighbor_call').distinct().count()}")

#import dataset
airport_codes_with_time_zones = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header=None)
column_namess = {0: 'AirportID', 1: 'Name', 2: 'City', 3: 'Country', 4: 'IATA', 5: 'ICAO', 6: 'airport_latitude', 
             7: 'airport_longitude', 8: 'airport_elevation', 9: 'Timezone', 10: 'Daylight_savings_time', 11: 'TZ_Timezone', 12: 'Type', 13: 'Source'}
 
#add column names 
airport_codes_with_time_zones.rename(columns=column_namess, inplace=True)
#selecting desired columns 
codes = airport_codes_with_time_zones[['Country','IATA','ICAO','Timezone', 'TZ_Timezone']]
 
# found one airport in the final dataset wiht invalid timezone
# filling the invalid timezone with the correct timezone
codes.loc[codes['IATA'] == 'BIH', 'TZ_Timezone'] = 'America/Los_Angeles' 
 
#converting to PySpark Dataframe
airport_codes = spark.createDataFrame(codes)
 
#filtering stations data set with airport_codes dataset
stations_data_filtered = df_stations.join(airport_codes).where(df_stations["neighbor_call"] == airport_codes["ICAO"])

 Total number of aiport codes :2229
/databricks/python/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/databricks/python/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [0]:
#filtering stations data set with airport_codes dataset and joining
stations_data_filtered = df_stations.join(airport_codes).where(df_stations["neighbor_call"] == airport_codes["ICAO"])

#investigating unique countries.
display(stations_data_filtered.groupBy('Country').count())

Country,count
United States,2881256
Puerto Rico,6711
Virgin Islands,4474
Germany,6711
Madagascar,2237
Zambia,2237
Belize,2237
Kenya,2237
Australia,2237
Canada,2237


In [0]:
#selecting US, Puerto Rico, and Virgin Islands.
countries =['United States','Puerto Rico','Virgin Islands']
stations_data_filtered_US = stations_data_filtered.filter(stations_data_filtered.Country.isin(countries))
 
#selecting desired columns
cols_to_keeep = ['station_id', 'neighbor_name', 'neighbor_state','neighbor_call','IATA',
                 'distance_to_neighbor','Country', 'Timezone', 'TZ_timezone']
 
stations_data_us = stations_data_filtered_US.select(cols_to_keeep)
 
#selecting weather stations that are the closest to each airport.
minimum_distance = stations_data_us.groupby('neighbor_call').agg(min('distance_to_neighbor'))
f_airport_stations = stations_data_us.join(minimum_distance, ['neighbor_call'])
 
#final station data by joing. 
station_final_df = f_airport_stations.filter(f_airport_stations['distance_to_neighbor'] == f_airport_stations['min(distance_to_neighbor)'])
display(station_final_df)
station_final_df.count()

neighbor_call,station_id,neighbor_name,neighbor_state,IATA,distance_to_neighbor,Country,Timezone,TZ_timezone,min(distance_to_neighbor)
KDWH,72242953910,HOOKS MEMORIAL AIRPT,TX,DWH,0.0,United States,-6,America/Chicago,0.0
KRCX,72375854928,RUSK COUNTY AIRPORT,WI,\N,0.0,United States,-5,\N,0.0
KOLV,72216563808,OLIVE BRANCH AIRPORT,MS,OLV,0.0,United States,-6,America/Chicago,0.0
KAMA,72363023047,AMARILLO RICK HUSBAND INTL AI,TX,AMA,0.0,United States,-6,America/Chicago,0.0
KGGG,72247003901,EAST TEXAS REGIONAL ARPT,TX,GGG,0.0,United States,-6,America/Chicago,0.0
KGTR,72330753893,GOLDEN TRIANGLE RGNL APT,MS,GTR,0.0,United States,-6,America/Chicago,0.0
KMTJ,72476593013,MONTROSE REGIONAL AIRPORT,CO,MTJ,0.0,United States,-7,America/Denver,0.0
KAIA,72563524044,ALLIANCE MUNICIPAL AIRPORT,NE,AIA,0.0,United States,-7,America/Denver,0.0
KRHI,72741504803,RHINELANDER-ONEIDA CO ARPT,WI,RHI,0.0,United States,-6,America/Chicago,0.0
KRSW,72210812894,SW FLORIDA INTN AIRPORT,FL,RSW,0.0,United States,-5,America/New_York,0.0


Out[8]: 1304

In [0]:
display(station_final_df.groupBy('IATA','TZ_timezone', 'neighbor_call' ).count())

IATA,TZ_timezone,neighbor_call,count
FRN,America/Anchorage,PAFR,1
DAL,America/Chicago,KDAL,1
AOO,America/New_York,KAOO,1
SKF,America/Chicago,KSKF,1
DOV,America/New_York,KDOV,1
PMD,America/Los_Angeles,KPMD,1
LNY,Pacific/Honolulu,PHNY,1
STG,America/Anchorage,PAPB,1
MIA,America/New_York,KMIA,1
FHU,America/Phoenix,KFHU,1


We have some values values as missig in IATA and TZ_timezone. We will revisit this once we perform a filter with airline data set. One joins have been completed, these null values will drop

In [0]:
# Aside from IATA and TZ_timezone having "\N" values, checking for nulls. 
df_Columns_s = station_final_df
station_final_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns_s.columns]
   ).toPandas()

,neighbor_call,station_id,neighbor_name,neighbor_state,IATA,distance_to_neighbor,Country,Timezone,TZ_timezone,min(distance_to_neighbor)
0,0,0,0,0,0,0,0,0,0,0


# Flight Dataset - EDA

For data cleanup for the airline dataset, please refer to the Data_cleanup_and_join databricks notebook.

In [0]:
display(airline_final_df)
print(f' Airline dataset: number of rows = {airline_final_df.count()}, number of columns =  {len(airline_final_df.columns)}')

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,DEP_DEL15,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,Origin_IATA,origin_Timezone,origin_TZ,dest_IATA,dest_Timezone,dest_TZ,Date_Time_sched_dep,Date_Time_sched_arrival,two_hrs_pre_flight,two_hrs_pre_flight_utc,Date_Time_sched_dep_utc,Date_Time_sched_arrival_utc
2015,1,3,28,6,2015-03-28,WN,19393,WN,N731SA,FLL,SJU,1220,1450,null,0.0,150.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-03-28 12:00:00,2015-03-28 14:00:00,2015-03-28 10:00:00,2015-03-28T14:00:00.000+0000,2015-03-28T16:00:00.000+0000,null
2019,4,11,30,6,2019-11-30,WN,19393,WN,N447WN,FLL,SJU,0855,1230,null,0.0,155.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2019-11-30 08:00:00,2019-11-30 12:00:00,2019-11-30 06:00:00,2019-11-30T11:00:00.000+0000,2019-11-30T13:00:00.000+0000,null
2015,1,1,17,6,2015-01-17,WN,19393,WN,N8621A,FLL,SJU,1705,2050,-40.0,0.0,165.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-01-17 17:00:00,2015-01-17 20:00:00,2015-01-17 15:00:00,2015-01-17T20:00:00.000+0000,2015-01-17T22:00:00.000+0000,null
2015,3,8,6,4,2015-08-06,WN,19393,WN,N8307K,FLL,SJU,1125,1420,-39.0,0.0,175.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-08-06 11:00:00,2015-08-06 14:00:00,2015-08-06 09:00:00,2015-08-06T13:00:00.000+0000,2015-08-06T15:00:00.000+0000,null
2015,3,7,8,3,2015-07-08,WN,19393,WN,N8614M,FLL,SJU,1125,1420,-38.0,0.0,175.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-07-08 11:00:00,2015-07-08 14:00:00,2015-07-08 09:00:00,2015-07-08T13:00:00.000+0000,2015-07-08T15:00:00.000+0000,null
2021,3,7,23,5,2021-07-23 00:00:00,WN,19393,WN,N8681M,FLL,SJU,1000,1250,-37.0,0.0,170.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2021-07-23 10:00:00,2021-07-23 12:00:00,2021-07-23 08:00:00,2021-07-23T12:00:00.000+0000,2021-07-23T14:00:00.000+0000,null
2015,4,11,25,3,2015-11-25,WN,19393,WN,N774SW,FLL,SJU,1220,1600,-36.0,0.0,160.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-11-25 12:00:00,2015-11-25 16:00:00,2015-11-25 10:00:00,2015-11-25T15:00:00.000+0000,2015-11-25T17:00:00.000+0000,null
2021,2,5,25,2,2021-05-25 00:00:00,WN,19393,WN,N8311Q,FLL,SJU,1115,1400,-36.0,0.0,165.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2021-05-25 11:00:00,2021-05-25 14:00:00,2021-05-25 09:00:00,2021-05-25T13:00:00.000+0000,2021-05-25T15:00:00.000+0000,null
2019,4,10,3,4,2019-10-03,WN,19393,WN,N8526W,FLL,SJU,1800,2040,-35.0,0.0,160.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2019-10-03 18:00:00,2019-10-03 20:00:00,2019-10-03 16:00:00,2019-10-03T20:00:00.000+0000,2019-10-03T22:00:00.000+0000,null
2015,1,2,6,5,2015-02-06,WN,19393,WN,N713SW,FLL,SJU,1220,1605,-35.0,0.0,165.0,1046.0,5,FLL,-5,America/New_York,null,null,null,2015-02-06 12:00:00,2015-02-06 16:00:00,2015-02-06 10:00:00,2015-02-06T15:00:00.000+0000,2015-02-06T17:00:00.000+0000,null


 Airline dataset: number of rows = 41551807, number of columns =  31


In [0]:
# There are some null values that will be dropped after joining
display(airline_final_df.groupBy('dest_IATA','dest_Timezone','dest_TZ','Date_Time_sched_arrival_utc').count())

dest_IATA,dest_Timezone,dest_TZ,Date_Time_sched_arrival_utc,count
RSW,-5,America/New_York,2018-07-04T02:00:00.000+0000,3
BWI,-5,America/New_York,2018-09-15T01:00:00.000+0000,17
FLL,-5,America/New_York,2017-07-17T23:00:00.000+0000,15
BWI,-5,America/New_York,2015-08-17T21:00:00.000+0000,16
MCO,-5,America/New_York,2020-05-09T21:00:00.000+0000,8
DTW,-5,America/New_York,2018-01-23T22:00:00.000+0000,10
ATL,-5,America/New_York,2020-06-30T23:00:00.000+0000,37
IND,-5,America/New_York,2015-03-11T23:00:00.000+0000,6
PVD,-5,America/New_York,2015-05-01T21:00:00.000+0000,3
TPA,-5,America/New_York,2016-08-16T12:00:00.000+0000,5


In [0]:
airline_final_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- QUARTER: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- DEP_DEL15: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- DISTANCE_GROUP: integer (nullable = true)
 |-- Origin_IATA: string (nullable = true)
 |-- origin_Timezone: string (nullable = true)
 |-- origin_TZ: string (nullable = true)
 |-- dest_IATA: string (nullable = true)
 |-- dest_Timezo

In [0]:
#The number of flights delayed. 
total = airline_final_df.count()
display(airline_final_df.groupBy('DEP_DEL15').count().withColumnRenamed('count', 'total_count').withColumn('Percentage', (f.col('total_count') / total) * 100))

DEP_DEL15,total_count,Percentage
0.0,34443478,82.89285228919165
1.0,7108329,17.10714771080834


Output can only be rendered in Databricks

There is a class imbalance in our output variable. We have about %17 of flights that are delayed and over %82 of flights that are non delayed.

In [0]:
# The number of delays vs non-delays over the years
total = airline_final_df.count()
display(airline_final_df.groupBy('YEAR','DEP_DEL15').count().withColumnRenamed('count', 'total_count').withColumn('Percentage', (f.col('total_count') / total) * 100))

YEAR,DEP_DEL15,total_count,Percentage
2021,1.0,1023479,2.463139569357357
2020,1.0,400792,0.9645597362348164
2015,0.0,4673695,11.247874250089772
2015,1.0,1055500,2.5402024032312243
2017,0.0,4579381,11.020894951692474
2019,1.0,1359573,3.2719948858060497
2016,0.0,4599572,11.06948730292283
2016,1.0,952225,2.2916572557241612
2017,1.0,1012546,2.4368278375956067
2018,1.0,1304214,3.138766022859126


Output can only be rendered in Databricks

It appears that the number of delays and non-delays are relatively stable. In 2020, we do see a huge dip in both due to the COVID lock downs.

In [0]:
#next we will investigate the Percentage of delays by airline
airline_delay_count_0 = airline_final_df.groupby('OP_CARRIER_AIRLINE_ID', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "1")
airline_delay_count_1 = airline_final_df.groupby('OP_CARRIER_AIRLINE_ID', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "0")
airline_delay_count_1 = airline_delay_count_1.selectExpr("OP_CARRIER_AIRLINE_ID as OP_CARRIER_AIRLINE_ID_1", "DEP_DEL15 as DEP_DEL15_1", "count as count_1")
airline_delay_count_0 = airline_delay_count_0.join(airline_delay_count_1,airline_delay_count_0.OP_CARRIER_AIRLINE_ID ==  airline_delay_count_1.OP_CARRIER_AIRLINE_ID_1,"inner")
airline_delay_count_0 = airline_delay_count_0.drop(airline_delay_count_0.OP_CARRIER_AIRLINE_ID_1)
airline_delay_count_0 = airline_delay_count_0.withColumn("relative_delay", (col("count_1") / (col("count")+col("count_1"))) * 100).orderBy(col('relative_delay').desc())
display(airline_delay_count_0)

OP_CARRIER_AIRLINE_ID,DEP_DEL15,count,DEP_DEL15_1,count_1,relative_delay
20409,0.0,1334029,1.0,425607,24.18721826559584
20436,0.0,588679,1.0,170233,22.431190968122785
21171,0.0,170809,1.0,46197,21.28835147415279
20368,0.0,313661,1.0,82717,20.868211656550063
19393,0.0,6709753,1.0,1720205,20.405854928340094
20416,0.0,885817,1.0,210339,19.188783348355525
19977,0.0,2944336,1.0,646475,18.003593060174985
20366,0.0,1429666,1.0,309673,17.80406234782294
20378,0.0,590693,1.0,123033,17.238127796941683
19805,0.0,4632185,1.0,954681,17.087952351103464


Output can only be rendered in Databricks

It appears that some airlines have more delays than others.

In [0]:
#Flight delay by airport
airport_delay_count_0 = airline_final_df.groupby('ORIGIN', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "1")
airport_delay_count_1 = airline_final_df.groupby('ORIGIN', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "0")
airport_delay_count_1 = airport_delay_count_1.selectExpr("ORIGIN as ORIGIN_1", "DEP_DEL15 as DEP_DEL15_1", "count as count_1")
airport_delay_count_0 = airport_delay_count_0.join(airport_delay_count_1,airport_delay_count_0.ORIGIN ==  airport_delay_count_1.ORIGIN_1,"inner")
airport_delay_count_0 = airport_delay_count_0.drop(airport_delay_count_0.ORIGIN_1)
airport_delay_count_0 = airport_delay_count_0.withColumn("total", col("count")+col("count_1"))
airport_delay_count_0 = airport_delay_count_0.withColumn("relative_delay", (col("count_1") / (col("count")+col("count_1"))) * 100).orderBy(col('relative_delay').desc()).take(20)
display(airport_delay_count_0)

ORIGIN,DEP_DEL15,count,DEP_DEL15_1,count_1,total,relative_delay
BIH,0.0,15,1.0,20,35,57.14285714285714
CDB,0.0,135,1.0,105,240,43.75
ADK,0.0,443,1.0,248,691,35.89001447178003
ILG,0.0,137,1.0,74,211,35.07109004739337
OGD,0.0,288,1.0,126,414,30.434782608695656
CEC,0.0,123,1.0,51,174,29.310344827586203
MMH,0.0,774,1.0,317,1091,29.055912007332722
OTH,0.0,1576,1.0,630,2206,28.558476881233002
UST,0.0,297,1.0,117,414,28.26086956521739
HGR,0.0,474,1.0,186,660,28.18181818181818


Output can only be rendered in Databricks

In [0]:
# Which 20 airports have the highest volume of flights?
display(airline_final_df.groupby('ORIGIN').count().orderBy(col('count').desc()).take(20))


ORIGIN,count
ATL,2452308
ORD,1876827
DFW,1682740
DEN,1581386
LAX,1360137
CLT,1149126
PHX,1090198
SFO,1024060
LAS,1009248
IAH,990745


Output can only be rendered in Databricks

In [0]:
#let's investigate the number of delays in the top of 20 airports by volume.

airport_delay_count_0 = airline_final_df.groupby('ORIGIN', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "1")
airport_delay_count_1 = airline_final_df.groupby('ORIGIN', 'DEP_DEL15').count().filter(airline_final_df.DEP_DEL15 != "0")
airport_delay_count_1 = airport_delay_count_1.selectExpr("ORIGIN as ORIGIN_1", "DEP_DEL15 as DEP_DEL15_1", "count as count_1")
airport_delay_count_0 = airport_delay_count_0.join(airport_delay_count_1,airport_delay_count_0.ORIGIN ==  airport_delay_count_1.ORIGIN_1,"inner")
airport_delay_count_0 = airport_delay_count_0.drop(airport_delay_count_0.ORIGIN_1)
airport_delay_count_0 = airport_delay_count_0.withColumn("total", col("count")+col("count_1"))
airport_delay_count_0 = airport_delay_count_0.withColumn("relative_delay", (col("count_1") / (col("count")+col("count_1"))) * 100).orderBy(col('relative_delay').desc())
airport_delay_count_00 = airport_delay_count_0.withColumn("relative_delay", col("count_1") / (col("count")+col("count_1"))).orderBy(col('total').desc()).take(20)
display(airport_delay_count_00)

ORIGIN,DEP_DEL15,count,DEP_DEL15_1,count_1,total,relative_delay
ATL,0.0,2060063,1.0,392245,2452308,0.15994932121087563
ORD,0.0,1502100,1.0,374727,1876827,0.19965985144075613
DFW,0.0,1349606,1.0,333134,1682740,0.1979711660743787
DEN,0.0,1272688,1.0,308698,1581386,0.19520724225457922
LAX,0.0,1098938,1.0,261199,1360137,0.19203874315602032
CLT,0.0,963238,1.0,185888,1149126,0.16176468028745325
PHX,0.0,902309,1.0,187889,1090198,0.17234392284704247
SFO,0.0,821149,1.0,202911,1024060,0.19814366345721932
LAS,0.0,810111,1.0,199137,1009248,0.19731225625416152
IAH,0.0,824905,1.0,165840,990745,0.1673891869249908


Output can only be rendered in Databricks

In [0]:
#Next let's take a look at day of the month, day of the month and day of the week and see if there are any insights we can gain. 

display(airlines_final_df.groupBy('YEAR','QUARTER','MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEP_DEL15').count())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2647101326239518> in <cell line: 3>()
      1 #Next let's take a look at day of the month, day of the month and day of the week and see if there are any insights we can gain.
      2 
----> 3 display(airlines_final_df.groupBy('YEAR','QUARTER','MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEP_DEL15').count())

NameError: name 'airlines_final_df' is not defined

Looking at the total number of delayed and non delayed on different timelines above, does not tell us much visually. Thera are minor difference in all timelines.

In [0]:
# Delay by Flight Distance

display(airlines_final_df.groupBy('DISTANCE_GROUP', 'DISTANCE', 'DEP_DEL15').count())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2647101326239520> in <cell line: 3>()
      1 # Delay by Flight Distance
      2 
----> 3 display(airlines_final_df.groupBy('DISTANCE_GROUP', 'DISTANCE', 'DEP_DEL15').count())

NameError: name 'airlines_final_df' is not defined

It appears that the vast majority of flights are between 251-749 miles. We can also that the flights that are between 251-749 miles have the lowest cancellation rate.

In [0]:
display(airlines_final_df.groupBy('origin_TZ', 'DEP_DEL15').count())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2647101326239522> in <cell line: 1>()
----> 1 display(airlines_final_df.groupBy('origin_TZ', 'DEP_DEL15').count())

NameError: name 'airlines_final_df' is not defined

It appears that most flights depart from a few major timezones

#Weather Dataset - EDA

For clean up of the weather data, refer to the Data_cleanup_and_join notebook

In [0]:
display(weather_final_df)
print(f' Weather dataset: number of rows = {weather_final_df.count()}, number of columns =  {len(weather_final_df.columns)}')

station_id,ELEVATION,NAME,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,IATA,Date_Time_utc
72205012815,27.4,"ORLANDO INTERNATIONAL AIRPORT, FL US",30.22,58,60,0.00,BR:1 ||,+0.04,8,93,BKN:07 7 OVC:08 23,30.22,30.11,5.00,59,360,null,8,MCO,2015-01-01T06:00:00.000+0000
72515614748,291.1,"ELMIRA CORNING REGIONAL AIRPORT, NY US",30.02,4,22,0.00,null,+0.06,6,46,OVC:08 50,30.06,29.00,10.00,17,240,29,17,ELM,2015-01-01T06:00:00.000+0000
72575024126,1362.5,"OGDEN HINKLEY AIRPORT, UT US",30.08,-1,13,0.00,null,null,null,54,CLR:00,30.21,25.56,10.00,10,VRB,null,6,OGD,2015-01-01T08:00:00.000+0000
72753014914,274.3,"FARGO HECTOR INTERNATIONAL AIRPORT, ND US",29.81,17,25,0.00,null,null,null,72,OVC:08 43,29.85,28.85,10.00,22,260,null,13,FAR,2015-01-01T07:00:00.000+0000
72351013966,310.0,"WICHITA FALLS MUNICIPAL AIRPORT, TX US",30.38,15,25,0.00,null,null,null,66,OVC:08 14,30.41,29.27,null,22,010,null,3,SPS,2015-01-01T08:00:00.000+0000
72204012838,8.2,"MELBOURNE INTERNATIONAL AIRPORT, FL US",30.19,67,68,0.05,-RA:02 BR:1 |RA |,null,null,96,BKN:07 5 OVC:08 19,30.19,30.16,5.00,67,360,null,7,MLB,2015-01-01T07:00:00.000+0000
72222313899,34.1,"PENSACOLA REGIONAL AIRPORT, FL US",30.32,43,45,0.00,BR:1 ||,null,null,93,OVC:08 4,30.33,30.19,6.00,44,360,null,8,PNS,2015-01-01T08:00:00.000+0000
72292703177,100.0,"CARLSBAD MCCLELLAN PALOMAR AIRPORT, CA US",30.07,28,39,0.00,null,null,null,65,CLR:00,30.07,29.72,10.00,35,070,null,6,CLD,2015-01-01T10:00:00.000+0000
72326013891,293.2,"KNOXVILLE AIRPORT, TN US",30.38,22,27,0.00,null,null,null,81,CLR:00,30.40,29.32,10.00,25,000,null,0,TYS,2015-01-01T07:00:00.000+0000
91182022521,2.1,"HONOLULU INTERNATIONAL AIRPORT, HI US",29.89,54,65,0.00,null,+0.02,8,68,OVC:08 55,29.89,29.87,10.00,59,010,null,6,HNL,2015-01-01T12:00:00.000+0000


 Weather dataset: number of rows = 23117906, number of columns =  21


In [0]:
#let's find the earliest and latest data from this dataset.
earliest_date_weather =weather_final_df.select(f.min(f.col("Date_Time_utc")).alias("MIN")).limit(1).collect()[0].MIN
latest_date_weather =weather_final_df.select(f.max(f.col("Date_Time_utc")).alias("MAX")).limit(1).collect()[0].MAX
print(f"Earliest data: {earliest_date_weather}")
print(f"Latest date: {latest_date_weather}")

Earliest data: 2015-01-01 05:00:00
Latest date: 2022-01-01 10:00:00


In [0]:
# Checking for Nulls
df_Columns_w = weather_final_df.select([c for c in weather_final_df.columns if c not in {'Date_Time_utc', 'station_id', 'NAME', 'IATA'}])
weather_final_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns_w.columns]).toPandas()

,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,0,20542,72205,44593,2769705,20109629,15958301,15958301,73944,278815,1856519,284794,37692,347251,80821,20362379,80267


Most columns have nulls and they will be dropped once the joins have been completed

In [0]:
weather_numerical_df = weather_final_df.select([c for c in weather_final_df.columns if c not in {'Date_Time_utc', 'station_id', 'NAME', 'IATA',
                                                                                                 'HourlyPresentWeatherType','HourlySkyConditions',
                                                                                                 'HourlyWindDirection','HourlyPressureChange'}])


weather_numerical_df = weather_numerical_df.withColumn("ELEVATION", weather_numerical_df["ELEVATION"].cast(IntegerType())) \
                                           .withColumn("HourlyAltimeterSetting", weather_numerical_df["HourlyAltimeterSetting"].cast(IntegerType())) \
                                           .withColumn("HourlyDewPointTemperature", weather_numerical_df["HourlyDewPointTemperature"].cast(IntegerType())) \
                                           .withColumn("HourlyDryBulbTemperature", weather_numerical_df["HourlyDryBulbTemperature"].cast(IntegerType())) \
                                           .withColumn("HourlyPrecipitation", weather_numerical_df["HourlyPrecipitation"].cast(IntegerType())) \
                                           .withColumn("HourlyPressureTendency", weather_numerical_df["HourlyPressureTendency"].cast(IntegerType())) \
                                           .withColumn("HourlyRelativeHumidity", weather_numerical_df["HourlyRelativeHumidity"].cast(IntegerType())) \
                                           .withColumn("HourlySeaLevelPressure", weather_numerical_df["HourlySeaLevelPressure"].cast(IntegerType())) \
                                           .withColumn("HourlyStationPressure", weather_numerical_df["HourlyStationPressure"].cast(IntegerType())) \
                                           .withColumn("HourlyVisibility", weather_numerical_df["HourlyVisibility"].cast(IntegerType())) \
                                           .withColumn("HourlyWetBulbTemperature", weather_numerical_df["HourlyWetBulbTemperature"].cast(IntegerType())) \
                                           .withColumn("HourlyWindGustSpeed", weather_numerical_df["HourlyWindGustSpeed"].cast(IntegerType())) \
                                           .withColumn("HourlyWindSpeed", weather_numerical_df["HourlyWindSpeed"].cast(IntegerType()))
                                            

display(weather_numerical_df)

ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindGustSpeed,HourlyWindSpeed
27,30,58,60,0,8,93,30,30,5,59,null,8
291,30,4,22,0,6,46,30,29,10,17,29,17
1362,30,-1,13,0,null,54,30,25,10,10,null,6
274,29,17,25,0,null,72,29,28,10,22,null,13
310,30,15,25,0,null,66,30,29,null,22,null,3
8,30,67,68,0,null,96,30,30,5,67,null,7
34,30,43,45,0,null,93,30,30,6,44,null,8
100,30,28,39,0,null,65,30,29,10,35,null,6
293,30,22,27,0,null,81,30,29,10,25,null,0
2,29,54,65,0,8,68,29,29,10,59,null,6


Output can only be rendered in Databricks

-From the numerical columns, it only appears that HourlyDewPointTemperature, HourlyDryBulbTemperature, and HourlyWetBulbTemperature are possibly normally ditributed.\
-HourlyPressureTendency and HourlyWindGustSpeed have the most number of nulls. \
-HourlyPrecipitation's most values are zero.